In [1]:
%pwd
%cd ../..

/home/rydevera3/data-science/umn_research/Deeplifting


In [2]:
import glob

import numpy as np
import pandas as pd

from deeplifting.problems import PROBLEMS_BY_NAME

In [ ]:
# Get the results from deeplifting
files = glob.glob('./results/results-2023-08-03-2*')
deeplifting_results = pd.read_parquet(files)

# Select specific columns for results
deeplifting_results = (
    deeplifting_results[['problem_name', 'algorithm', 'x1', 'x2', 'f', 'total_time', 'global_minimum']]
)

# For now let's remove sine-envelope function because there is something
# going on there
mask = deeplifting_results['problem_name'] != 'sine_envelope'
deeplifting_results = deeplifting_results.loc[mask].reset_index(drop=True)
deeplifting_results['hits'] = np.abs(
    deeplifting_results['global_minimum'] - deeplifting_results['f']
) <= 1e-4
deeplifting_results['hits'] = deeplifting_results['hits'].astype(int)
deeplifting_results = deeplifting_results.rename(columns={'total_time': 'time'})
deeplifting_results = deeplifting_results.drop(columns=['global_minimum'])
deeplifting_results.head()

In [ ]:
deeplifting_results.groupby('problem_name')['hits'].mean()

In [64]:
# Load in the comparisons
comparison_files = glob.glob('./results/algorithm-comparisons-*.parquet')
comparison_results = pd.read_parquet(comparison_files)

# # Omit sine envelope function
# mask = comparison_results['problem_name'] != 'sine_envelope'
# comparison_results = comparison_results.loc[mask].reset_index(drop=True)
problems = [
    'ackley',
    'ackley2',
    'adjiman',
    'alpine1',
    'alpine2',
    'bartels_conn',
    'beale',
    'bird',
    'bohachevsky3',
    'booth',
    'bukin_n2',
    'bukin_n4',
    'bukin_n6',
    'camel_3hump',
    'camel_6hump',
    'chung_reynolds',
    'cube',
    'cross_in_tray',
    'cross_leg_table',
    'drop_wave',
    'eggholder',
    'griewank',
    'holder_table',
    'levy',
    'levy_n13',
    'rastrigin',
    'schaffer_n2',
    'schaffer_n4',
    'schwefel',
    'shubert',
    'ex8_1_1',
    'kriging_peaks_red010',
    'mathopt6',
    'quantum',
    'rosenbrock',
]

comparison_results = comparison_results.loc[comparison_results['problem_name'].isin(problems)]

In [65]:
comparison_results['problem_name'].value_counts()

problem_name
levy                    80
griewank                80
schaffer_n4             80
mathopt6                80
eggholder               80
bukin_n6                80
cube                    80
ex8_1_1                 80
holder_table            80
shubert                 80
schwefel                80
adjiman                 80
cross_in_tray           80
bukin_n4                80
drop_wave               80
alpine1                 80
camel_6hump             80
bartels_conn            80
alpine2                 80
bird                    80
schaffer_n2             80
quantum                 80
rosenbrock              80
bukin_n2                80
ackley                  80
kriging_peaks_red010    80
beale                   80
camel_3hump             80
cross_leg_table         80
bohachevsky3            80
chung_reynolds          80
rastrigin               80
booth                   80
levy_n13                80
ackley2                 80
Name: count, dtype: int64

In [66]:
results_df = comparison_results.copy()

In [67]:
results_df.loc[results_df['problem_name'] == 'ackley3']

,x1,x2,f,algorithm,time,problem_name,hits


In [68]:
aggregated_results_df = (
    results_df
    .groupby(['problem_name', 'algorithm'])
    .agg({'hits': 'mean'})
    .rename(columns={'hits': 'Success Rate'})
    .unstack()
    .transpose()
    .reorder_levels(['algorithm', None])
    .transpose()
)

In [69]:
aggregated_results_df.mean().sort_values(ascending=False)

algorithm                           
Dual Annealing          Success Rate    0.831429
Differential Evolution  Success Rate    0.778571
IPOPT                   Success Rate    0.420000
PyGRANSO                Success Rate    0.397143
dtype: float64

In [70]:
aggregated_results_df

algorithm,Differential Evolution,Dual Annealing,IPOPT,PyGRANSO
,Success Rate,Success Rate,Success Rate,Success Rate
problem_name,,,,
ackley,1.00,1.00,0.05,0.00
ackley2,0.90,1.00,1.00,0.05
adjiman,1.00,1.00,0.95,0.85
alpine1,1.00,1.00,0.95,0.75
alpine2,0.00,0.00,0.00,0.00
bartels_conn,0.60,1.00,0.70,0.95
beale,0.85,1.00,0.70,0.55
bird,0.70,1.00,0.50,0.40


In [71]:
aggregated_results_df.shape

(35, 4)

In [ ]:
file = [
    './results/results-2023-08-2-layer-512-holder_table-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-512-mathopt6-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-schwefel-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-shubert-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-512-cross_leg_table-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-ackley2-0-particle-leaky_relu.parquet',
]
df = pd.read_parquet(file)

In [ ]:
df['hits'] = np.abs(df['global_minimum'] - df['f']) <= 1e-4

In [ ]:
df.groupby('problem_name')['hits'].agg(['mean', 'count'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def ackley3(x, y):
    return -200 * np.exp(-0.02 * np.sqrt(x ** 2 + y ** 2)) + 5 * np.exp(np.cos(3 * x) + np.sin(3 * y))

x = np.linspace(-32, 32, 10000)
y = np.linspace(-32, 32, 10000)
X, Y = np.meshgrid(x, y)
Z = ackley3(X, Y)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, cmap='viridis')

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_title('3D Plot of Ackley Function')

plt.show()

In [ ]:
ackley3(0, -0.40)

In [ ]:
np.min(Z)

In [54]:
problem = PROBLEMS_BY_NAME['cube']

In [55]:
problem

{'objective': <function deeplifting.problems.cube(x, results, trial, version='numpy')>,
 'bounds': [(-10, 10), (-10, 10)],
 'max_iterations': 1000,
 'global_minimum': 0,
 'dimensions': 2}

In [56]:
import numpy as np
import torch

In [57]:
results = np.zeros((1, 1, 3))
trial = 0
f = problem['objective']

In [62]:
values = [1.0, 1.0]
f(torch.tensor(values), results=results, trial=trial, version='pytorch')

tensor(0.)

In [63]:
f(np.array(values), results=results, trial=trial, version='numpy')

0.0